### RandomForest Feature Importance for Selection

Process steps:
* Build full dataset with decoys and compounds
* Build compound-only dataset
* Preprocess categorical with one-hot-encoding
* Find intersecting columns and filter compound-only to these
* Add non-linear transformations to compound-only dataset
* Create binary target with IC50 <= 10 as 1, else 0
* Using compound-only df, fine-tune random forest with GridSearchCV (CV = count of positive class)
* Extract feature importance and remove any with importance of 0
* Create important features in full dataset

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skewtest
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix, log_loss
from custom_functions import *
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import itertools as it
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import RandomUnderSampler

In [2]:
avail_transformations = ["log", "log2", "log10", "cubert", 
                         "sqrt", "exp", "exp2", "cube", "sq"]

* Build full-dataset
* Build compound-dataset
* Preprocess data adding one-hot-encoded features for both
* Find intersecting columns
* Add non-linear transformations and drop na's

In [3]:
# Load in full dataset
full_x, full_y = load_full_dataset()
# Preprocess variables
full_x = preprocess_variables(full_x)
# Extract list of available columns
full_columns = full_x.columns
print("Loading in compound dataset....")
# Read in compound dataset
compound_x, compound_y = load_compound_dataset()
# Preprocess
compound_x = preprocess_variables(compound_x)
# Find intersecting features
avail_columns = compound_x.columns.intersection(full_columns)
# Select features on subset
x_data = compound_x.loc[:, avail_columns]
y_data = compound_y.copy()
print("Adding non-linear features to compound dataset....")
# Add all transformations on compound data
for feature in x_data.columns[x_data.dtypes == 'float64']:
    x_data = add_transformations(x_data, feature)
# Drop any new columns with NaN due to improper transformation
x_data.replace([np.inf, -np.inf], np.nan, inplace=True)
x_data.dropna(axis=1, inplace=True)
assert not sum(x_data.isna().sum()), "Unexpected nulls found"

Adding Akt1_decoys_padel.csv....
Adding AmpC_decoys_padel.csv....
Adding cp3a4_decoys_padel.csv....
Adding cxcr4_decoys_padel.csv....
Adding gcr_decoys_padel.csv....
Adding HIVpr_decoys_padel.csv....
Adding HIVrt_decoys_padel.csv....
Adding Kif11_decoys_padel.csv....
Loading in compound dataset....
Adding non-linear features to compound dataset....


In [4]:
# Create binary variable
y_class = np.squeeze([int(y_val <= 10) for y_val in y_data])

### Exhaustive Hyperparameter Tuning
* SMOTE and resampling transformations on train only

In [14]:
# Search for best params with SMOTE on train
params = {"a_SMOTE_k_neighbors": [3, 4],
          "b_SMOTE_m_neighbors": [10],
          "c_model_n_estimators": [40, 50, 60],
          "d_model_learning_rate": [0.05],
          "smote": [True],
          "downsample": [False, 1, 3]}  # Bigger=>less downsampling
all_names = sorted(params)  # TODO remove sorted but ensure correct order
combinations = it.product(*(params[Name] for Name in all_names))

# Initialize params
best_score = -np.inf
best_params = None

for param_values in combinations:
    param_dict = dict(zip(all_names, param_values))

    model = AdaBoostClassifier(random_state=0, 
                               learning_rate=param_dict.get("d_model_learning_rate"),
                               n_estimators=param_dict.get("c_model_n_estimators"))
    smote = SMOTE(random_state=0, k_neighbors=param_dict.get("a_SMOTE_k_neighbors"),
                  m_neighbors=param_dict.get("b_SMOTE_m_neighbors"))
    kfold = StratifiedKFold(n_splits=sum(y_class), shuffle=True, random_state=0)
    prediction_df = pd.DataFrame(columns=["prediction"])
    scores = []
    for train, test in kfold.split(x_data, y_class):
        # Split into train/test
        x_train, y_train = x_data.iloc[train], y_class[train]
        x_test, y_test = x_data.iloc[test], y_class[test]
        assert sum(y_test) == 1, "Ensure only one positive class is in test per iteration"
        # Perform SMOTE transformation on train
        if param_dict.get("smote"):
            x_train, y_train = smote.fit_sample(x_train, y_train)
        # Downsample randomly
        if param_dict.get("downsample"):
            negative_n = len(y_train) - sum(y_train)
            positive_n = sum(y_train)
            balance = min(1, param_dict.get("downsample") * (positive_n / negative_n))
            negative_n *= balance
            negative_n = int(negative_n)
            down_sampler = RandomUnderSampler(ratio={0: negative_n, 1: positive_n}, random_state=0)
            x_train, y_train = down_sampler.fit_sample(x_train, y_train)
        model.fit(x_train, y_train)
        prediction = model.predict(x_test)
        # sample_weight = compute_sample_weight("balanced", y_train)
        scores.append(roc_auc_score(y_test, prediction))
    # Calculate scores for parameters
    params_score = np.mean(scores)
    # Test new score against best
    if params_score > best_score:
        # Store the best
        best_params = param_dict
        best_score = params_score

# Print the best params and score
print("Best Params:")
print(best_params)
print("Best Score:")
print(best_score)

Best Params:
{'a_SMOTE_k_neighbors': 4, 'b_SMOTE_m_neighbors': 10, 'c_model_n_estimators': 50, 'd_model_learning_rate': 0.05, 'downsample': False, 'smote': True}
Best Score:
0.6666666666666667


Best Params:
{'a_SMOTE_k_neighbors': 2, 'b_SMOTE_m_neighbors': 9, 'c_model_n_estimators': 35, 'd_model_learning_rate': 0.075, 'downsample': False, 'smote': False}
Best Score:
0.7424242424242425

### AdaBoostClassifier Implementation

In [6]:
# Implement LOPOCV (leave one Positive out cross val)
model = AdaBoostClassifier(random_state=0, 
                           learning_rate=param_dict.get("d_model_learning_rate"),
                           n_estimators=param_dict.get("c_model_n_estimators"))
smote = SMOTE(random_state=0, k_neighbors=param_dict.get("a_SMOTE_k_neighbors"),
              m_neighbors=param_dict.get("b_SMOTE_m_neighbors"))
kfold = StratifiedKFold(n_splits=sum(y_class), shuffle=True, random_state=0)
prediction_df = pd.DataFrame(columns=["prediction"])
for train, test in kfold.split(x_data, y_class):
    # Split into train/test
    x_train, y_train = x_data.iloc[train], y_class[train]
    x_test, y_test = x_data.iloc[test], y_class[test]
    assert sum(y_test) == 1, "Ensure only one positive class is in test per iteration"
    # Perform SMOTE transformation on train
    if param_dict.get("smote"):
        x_train, y_train = smote.fit_sample(x_train, y_train)
    # Downsample randomly
    if param_dict.get("downsample"):
        negative_n = len(y_train) - sum(y_train)
        positive_n = sum(y_train)
        balance = min(1, param_dict.get("downsample") * (positive_n / negative_n))
        negative_n *= balance
        negative_n = int(negative_n)
        down_sampler = RandomUnderSampler(ratio={0: negative_n, 1: positive_n}, random_state=0)
        x_train, y_train = down_sampler.fit_sample(x_train, y_train)
    model.fit(x_train, y_train)
    prediction = model.predict(x_test)
    # sample_weight = compute_sample_weight("balanced", y_train)
    prediction = model.predict(x_test)
    prediction_df = prediction_df.append(pd.DataFrame({"prediction": prediction}, index=test))
    prediction_df.sort_index(inplace=True)
    prediction_df = prediction_df.astype(int)
# Calculate scores for parameters
params_score = np.mean(scores)


### AdaBoostClassifier

In [8]:
from sklearn.ensemble import AdaBoostClassifier
# How well does AdaBoost predict potency?
print("Tuning AdaBoost on compound dataset....")
model = AdaBoostClassifier(random_state=0)
params = {"n_estimators": [35, 40, 45],
          "learning_rate": [0.05, 0.075]}
grid = GridSearchCV(estimator=model, param_grid=params, cv=5, n_jobs=3)

grid.fit(x_data, y_class)
print(grid.best_params_)
best_model = grid.best_estimator_

Tuning AdaBoost on compound dataset....
{'learning_rate': 0.05, 'n_estimators': 40}


In [9]:
# Analyze CV prediction performance
predict = cross_val_predict(
    best_model, x_data, y_class, cv=sum(y_class), method="predict")

predict_proba = cross_val_predict(
    best_model, x_data, y_class, cv=sum(y_class), method="predict_proba")

print(confusion_matrix(y_class, predict, labels=[1, 0]))
print(np.array([["TP", "FN"], ["FP", "TN"]]))


[[ 6  5]
 [ 5 31]]
[['TP' 'FN']
 ['FP' 'TN']]


In [20]:
# AdaBoost with SMOTE
!pip install -U imbalanced-learn
from imblearn.over_sampling import SMOTE



  Using cached https://files.pythonhosted.org/packages/80/a4/900463a3c0af082aed9c5a43f4ec317a9469710c5ef80496c9abc26ed0ca/imbalanced_learn-0.3.3-py3-none-any.whl
Requirement not upgraded as not directly required: scikit-learn in c:\miniconda3\envs\deep-learn\lib\site-packages (from imbalanced-learn) (0.19.1)
Requirement not upgraded as not directly required: numpy in c:\miniconda3\envs\deep-learn\lib\site-packages (from imbalanced-learn) (1.14.3)
Requirement not upgraded as not directly required: scipy in c:\miniconda3\envs\deep-learn\lib\site-packages (from imbalanced-learn) (1.0.1)


tensorflow-tensorboard 1.5.1 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorflow-tensorboard 1.5.1 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.
tensorboard 1.6.0 has requirement bleach==1.5.0, but you'll have bleach 2.1.3 which is incompatible.
tensorboard 1.6.0 has requirement html5lib==0.9999999, but you'll have html5lib 1.0.1 which is incompatible.


In [21]:
# Analyze CV Performance
print(pd.DataFrame(
    {"IC50": y_data, 
     "y_class": y_class, 
     "Prediction": predict, 
     "Proba": predict_proba[:,1]})[
    ["IC50", "y_class", "Prediction", "Proba"]])

       IC50  y_class  Prediction     Proba
0     0.036        1           1  0.999833
1    10.000        1           1  0.020414
2    50.000        0           0  0.012971
3    50.000        0           0  0.938043
4    50.000        0           0  0.012011
5     8.000        1           1  0.999862
6    50.000        0           0  0.012011
7    50.000        0           0  0.020414
8    35.000        0           0  0.020414
9    50.000        0           0  0.024974
10   45.000        0           0  0.020414
11   45.000        0           0  0.990988
12   40.000        0           0  0.011972
13   50.000        0           0  0.011972
14   50.000        0           0  0.011972
15   50.000        0           0  0.005838
16   50.000        0           0  0.005838
17   25.000        0           0  0.005838
18   50.000        0           0  0.005814
19   50.000        0           0  0.005814
20   15.000        0           0  1.000000
21    1.700        1           1  0.005838
22   10.000

In [11]:
# Analyze train prediction performance
best_model.fit(x_data, y_class)
predict = best_model.predict(x_data)
print(confusion_matrix(y_class, predict, labels=[1, 0]))
print(np.array([["TP", "FN"], ["FP", "TN"]]))

[[11  0]
 [ 0 36]]
[['TP' 'FN']
 ['FP' 'TN']]


Analyze feature importance

In [18]:
best_model.fit(x_data, y_class)
feat_importance = best_model.feature_importances_
best_features = [
    (f, i) for i, f in sorted(zip(feat_importance, x_data.columns), 
                              reverse=True) if i != 0]
feat_df = pd.DataFrame(best_features)
feat_df.columns = ["Feature", "Importance"]
feat_df.head(20)

,Feature,Importance
0,MATS5p_sq,0.175
1,MATS5m,0.150
2,ATSC6v,0.125
3,ATSC6v_sq,0.100
4,MATS5p_cube,0.075
5,MATS5m_exp2,0.075
6,MATS5m_exp,0.075
7,MATS5m_cube,0.075
8,MATS5p_exp2,0.025
9,MATS5p_exp,0.025


Descriptors Info
http://www.talete.mi.it/products/dragon_molecular_descriptor_list.pdf